In [5]:
import subprocess
import time
import os
from my_MRI_tool_kit import DirectoryReader_dict
from tqdm import tqdm


# 基本参数设置
MNI152 = 'icbm_avg_152_t1_tal_nlin_symmetric_VI.nii.gz'
antsRegistration= os.path.join('ANTs','antsRegistration.exe')
antsApplyTransforms= os.path.join('ANTs','antsApplyTransforms.exe')


# ███████████████
# shell命令运行
# ███████████████
def run_command(command, message=''):
    start_time = time.time()
    if message!='':
        print(message)
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    end_time = time.time()
    duration = end_time - start_time
    
    # 检查命令的返回值
    if result.returncode == 0:
        print(f'命令执行完毕,耗时{duration:.2f}秒')
    else:
        print(f"命令执行失败,耗时{duration:.2f}秒")
    

# ███████████████
# 找到最适合打头阵的序列，返回两个列表
# ███████████████
def find_which_first(all_files):
    # 按顺序找到最好的序列
    def select_target_file(files_list):
        priority = ['T1CE.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', 'fDWI_2.nii.gz', 'ADC.nii.gz', 'ADC_manual.nii.gz']

        for file in priority:
            if file in files_list:
                return file
        return None
    
    target_file = ''
    target_file = select_target_file(all_files)
    remaining_files =  []
    remaining_files = [file for file in all_files if file != target_file]

    print("先配准:", target_file)
    print("后配准:", remaining_files)
    
    # target_file是一个文件，里面必然有东西且只有一个；remaining_files是文件的列表，可以为空列表
    return [target_file,remaining_files]   


# ███████████████
# 找到最适合的配对，返回两个列表
# ███████████████
def find_which_best_patch(all_files, priority):
    # 按顺序找到最好的序列
    def select_target_file(files_list):
        # priority = ['T1CE.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', 'fDWI_2.nii.gz', 'ADC.nii.gz', 'ADC_manual.nii.gz']

        for file in priority:
            if file in files_list:
                return file
        return None
    
    target_file = ''
    target_file = select_target_file(all_files)
    remaining_files =  []
    remaining_files = [file for file in all_files if file != target_file]

    # target_file是一个文件，里面必然有东西且只有一个；remaining_files是文件的列表，可以为空列表
    return [target_file,remaining_files]   


# ███████████████
# 单个MRI的刚体配准
# ███████████████
def rigid_regisitration(template_file, input_file, output_file):
    # 相对路径
    work_dir = os.path.dirname(input_file)
    
    # 放射变换文件的前缀
    mat_prefix = os.path.join(work_dir,'rigidTemp_')
    mat_name = mat_prefix+'0GenericAffine.mat'
    
    # 一阶段
    command_Rigid = ' '.join([
        antsRegistration,'-d 3',
        '-m',f'MI[{template_file},{input_file},1,32,Regular,0.25]','-w [0.005, 0.995]','--float','-v',
        '-t','Rigid[0.1]',
        '-o',mat_prefix,
        '-c [1000x500x250x100x0,1e-6,10]','-s 4x3x2x1x1vox','-f 12x8x4x2x1',
    ])

    command_trans =  ' '.join([
        antsApplyTransforms,'-d 3','-n','Linear','-v',
        '-i',input_file,
        '-r',template_file,
        '-t',mat_name,
        '-o',output_file,
    ])

    run_command(command_Rigid,f'Rigid: {input_file}→{template_file}\n→{output_file}')
    run_command(command_trans,'')
    
    # 删除临时的变换文件
    os.remove(mat_name)

# 将所有初始序列先重采样，后对其

In [6]:
# 找到对应文件夹里所有文件
root_path = r"G:\Aim_new\JSPH_adc2fdwi"
max_depth = 3
directory_reader = DirectoryReader_dict(root_path, max_depth)
directories = directory_reader.get_files_dict()
print(len(directories))
directories

35


{'G:\\Aim_new\\JSPH_adc2fdwi\\X0633653\\20150430-Pre': ['ADC.nii.gz',
  'ADC_manual.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_ADC_manual.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz',
  '__rsmp_T2Flair.nii.gz'],
 'G:\\Aim_new\\JSPH_adc2fdwi\\X0815708\\20150313-Pre': ['ADC.nii.gz',
  'ADC_manual.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_ADC_manual.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz',
  '__rsmp_T2Flair.nii.gz'],
 'G:\\Aim_new\\JSPH_adc2fdwi\\X0819214\\20150324-Pre': ['ADC.nii.gz',
  'ADC_manual.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_ADC_manual.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1

In [7]:
def one_click(folder):
    # 先分出先后批次
    aaaa = find_which_first(os.listdir(folder))
    forerunner = aaaa[0]
    latecomer = aaaa[1]
    # 完整路径
    forerunner_full_path = os.path.join(folder,forerunner)
    latecomer_full_path = []
    for i in latecomer:
        latecomer_full_path.append(os.path.join(folder,i))
    
    # ————————————————————————————————
    # 先将 forerunner 重采样以及刚体配准到MNI152上
    rsmp_forerunner = os.path.join(os.path.dirname(forerunner_full_path),
                           '__rsmp_'+os.path.basename(forerunner_full_path))
    rigid_regisitration(MNI152, forerunner_full_path, rsmp_forerunner)   # template_file, input_file, output_file
    
    # ————————————————————————————————
    # 把其他所有序列都重采样到 rsmp_forerunner 上(互相配准，刚体)
    if len(latecomer_full_path)>0:
        for input_file in latecomer_full_path:
            output_file = os.path.join(os.path.dirname(input_file),
                       '__rsmp_'+os.path.basename(input_file))
            rigid_regisitration(rsmp_forerunner,input_file,output_file)   # template_file, input_file, output_file
    
        
# ————————————————————————————————
# 开始
al_folders = list(directories.keys())

for i in tqdm(range(len(al_folders))):
    folder = al_folders[i]
    print(f"{'#'*80}\n{folder}")
    
    one_click(folder)

  0%|                                                                                           | 0/35 [00:00<?, ?it/s]

################################################################################
G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre
先配准: T1CE.nii.gz
后配准: ['ADC.nii.gz', 'ADC_manual.nii.gz', 'fDWI_2.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_ADC_manual.nii.gz', '__rsmp_fDWI_2.nii.gz', '__rsmp_T1.nii.gz', '__rsmp_T1CE.nii.gz', '__rsmp_T2.nii.gz', '__rsmp_T2Flair.nii.gz']
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\T1CE.nii.gz→icbm_avg_152_t1_tal_nlin_symmetric_VI.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\__rsmp_T1CE.nii.gz
命令执行完毕,耗时4.41秒
命令执行完毕,耗时3.03秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\__rsmp_T1CE.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时3.25秒
命令执行完毕,耗时1.55秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0633653\20150430-Pre\__rsmp_T1CE.nii.gz
→G:\Aim_new\JSPH_adc

  3%|██▎                                                                                | 1/35 [01:17<43:52, 77.43s/it]

命令执行完毕,耗时2.29秒
################################################################################
G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre
先配准: T1CE.nii.gz
后配准: ['ADC.nii.gz', 'ADC_manual.nii.gz', 'fDWI_2.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_ADC_manual.nii.gz', '__rsmp_fDWI_2.nii.gz', '__rsmp_T1.nii.gz', '__rsmp_T1CE.nii.gz', '__rsmp_T2.nii.gz', '__rsmp_T2Flair.nii.gz']
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\T1CE.nii.gz→icbm_avg_152_t1_tal_nlin_symmetric_VI.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\__rsmp_T1CE.nii.gz
命令执行完毕,耗时5.23秒
命令执行完毕,耗时2.80秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\__rsmp_T1CE.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时2.79秒
命令执行完毕,耗时1.24秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0815708\20150313-Pre\__rsmp_T1CE.nii.gz
→G:\A

  6%|████▋                                                                              | 2/35 [02:43<45:20, 82.45s/it]

...

命令执行完毕,耗时2.56秒
################################################################################
G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre
先配准: T1CE.nii.gz
后配准: ['ADC.nii.gz', 'ADC_manual.nii.gz', 'fDWI_2.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_ADC_manual.nii.gz', '__rsmp_fDWI_2.nii.gz', '__rsmp_T1.nii.gz', '__rsmp_T1CE.nii.gz', '__rsmp_T2.nii.gz', '__rsmp_T2Flair.nii.gz']
Rigid: G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\T1CE.nii.gz→icbm_avg_152_t1_tal_nlin_symmetric_VI.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\__rsmp_T1CE.nii.gz
命令执行完毕,耗时8.90秒
命令执行完毕,耗时3.69秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\__rsmp_T1CE.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时3.12秒
命令执行完毕,耗时1.96秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X1095122\20200214-Pre\__rsmp_T1CE.nii.gz
→G:\A

 54%|████████████████████████████████████████████▌                                     | 19/35 [25:25<22:19, 83.73s/it]

命令执行完毕,耗时2.42秒
################################################################################
G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre
先配准: T1CE.nii.gz
后配准: ['ADC.nii.gz', 'ADC_manual.nii.gz', 'fDWI_2.nii.gz', 'T1.nii.gz', 'T2.nii.gz', 'T2Flair.nii.gz', '__rsmp_ADC.nii.gz', '__rsmp_ADC_manual.nii.gz', '__rsmp_fDWI_2.nii.gz', '__rsmp_T1.nii.gz', '__rsmp_T1CE.nii.gz', '__rsmp_T2.nii.gz', '__rsmp_T2Flair.nii.gz']
Rigid: G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\T1CE.nii.gz→icbm_avg_152_t1_tal_nlin_symmetric_VI.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\__rsmp_T1CE.nii.gz
命令执行完毕,耗时3.11秒
命令执行完毕,耗时2.45秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\__rsmp_T1CE.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时2.07秒
命令执行完毕,耗时1.40秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X1103838\20200430-Pre\__rsmp_T1CE.nii.gz
→G:\A

 54%|████████████████████████████████████████████▌                                     | 19/35 [26:37<22:25, 84.10s/it]


KeyboardInterrupt: 

# 注意 ADC 和 fDWI_2 最好和T2或T2Flair再进行一次配准

In [6]:
# 找到对应文件夹里所有文件
root_path = r"G:\Aim_new\JSPH_adc2fdwi"
max_depth = 3
directory_reader = DirectoryReader_dict(root_path, max_depth)
directories = directory_reader.get_files_dict()
print(len(directories))
directories

10


{'G:\\Aim_new\\JSPH_adc2fdwi\\X0841033\\20151010-Pre': ['ADC.nii.gz',
  'ADC_manual.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_ADC_manual.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz',
  '__rsmp_T2Flair.nii.gz'],
 'G:\\Aim_new\\JSPH_adc2fdwi\\X0844341\\20151103-Pre': ['ADC.nii.gz',
  'ADC_manual.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_ADC_manual.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1CE.nii.gz',
  '__rsmp_T2.nii.gz',
  '__rsmp_T2Flair.nii.gz'],
 'G:\\Aim_new\\JSPH_adc2fdwi\\X0849939\\20151220-Pre': ['ADC.nii.gz',
  'ADC_manual.nii.gz',
  'fDWI_2.nii.gz',
  'T1.nii.gz',
  'T1CE.nii.gz',
  'T2.nii.gz',
  'T2Flair.nii.gz',
  '__rsmp_ADC.nii.gz',
  '__rsmp_ADC_manual.nii.gz',
  '__rsmp_fDWI_2.nii.gz',
  '__rsmp_T1.nii.gz',
  '__rsmp_T1

In [7]:
for folder,seqs in tqdm(directories.items()):
    pair_sequence = {}
    
    # 有fDWI就肯定有ADC
    if '__rsmp_fDWI_2.nii.gz' in seqs:
        patten = ['__rsmp_T2Flair.nii.gz', '__rsmp_T2.nii.gz', '__rsmp_T1CE.nii.gz', '__rsmp_T1.nii.gz']
        best_sequence = find_which_best_patch(seqs, patten)[0]
        pair_sequence.update({'__rsmp_fDWI_2.nii.gz':best_sequence})
        
        pair_sequence.update({'__rsmp_ADC.nii.gz':'__rsmp_fDWI_2.nii.gz'}) 
        pair_sequence.update({'__rsmp_ADC_manual.nii.gz':'__rsmp_fDWI_2.nii.gz'}) 
        
    print(pair_sequence)
    for key,value in pair_sequence.items():
        input_file = os.path.join(folder,key)
        template_file = os.path.join(folder,value)
        try:
            rigid_regisitration(template_file, input_file, input_file)   # template_file, input_file, output_file
        except:
            pass

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

{'__rsmp_fDWI_2.nii.gz': '__rsmp_T2Flair.nii.gz', '__rsmp_ADC.nii.gz': '__rsmp_fDWI_2.nii.gz', '__rsmp_ADC_manual.nii.gz': '__rsmp_fDWI_2.nii.gz'}
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_fDWI_2.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_T2Flair.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_fDWI_2.nii.gz
命令执行完毕,耗时3.73秒
命令执行完毕,耗时2.65秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时3.66秒
命令执行完毕,耗时1.54秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0841033\20151010-Pre\__rsmp_ADC_manual.nii.gz
命令执行完毕,耗时3.76秒


 10%|████████▎                                                                          | 1/10 [00:16<02:32, 16.99s/it]

命令执行完毕,耗时1.59秒
{'__rsmp_fDWI_2.nii.gz': '__rsmp_T2Flair.nii.gz', '__rsmp_ADC.nii.gz': '__rsmp_fDWI_2.nii.gz', '__rsmp_ADC_manual.nii.gz': '__rsmp_fDWI_2.nii.gz'}
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_fDWI_2.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_T2Flair.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_fDWI_2.nii.gz
命令执行完毕,耗时3.81秒
命令执行完毕,耗时2.36秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时3.76秒
命令执行完毕,耗时1.76秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0844341\20151103-Pre\__rsmp_ADC_manual.nii.gz
命令执行完毕,耗时3.61秒


 20%|████████████████▌                                                                  | 2/10 [00:33<02:15, 16.95s/it]

...

命令执行完毕,耗时1.79秒
{'__rsmp_fDWI_2.nii.gz': '__rsmp_T2Flair.nii.gz', '__rsmp_ADC.nii.gz': '__rsmp_fDWI_2.nii.gz', '__rsmp_ADC_manual.nii.gz': '__rsmp_fDWI_2.nii.gz'}
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_fDWI_2.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_T2Flair.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_fDWI_2.nii.gz
命令执行完毕,耗时3.47秒
命令执行完毕,耗时2.60秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时2.99秒
命令执行完毕,耗时1.81秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0867493\20160429-Pre\__rsmp_ADC_manual.nii.gz
命令执行完毕,耗时3.79秒


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:41<00:18, 18.12s/it]

命令执行完毕,耗时1.61秒
{'__rsmp_fDWI_2.nii.gz': '__rsmp_T2Flair.nii.gz', '__rsmp_ADC.nii.gz': '__rsmp_fDWI_2.nii.gz', '__rsmp_ADC_manual.nii.gz': '__rsmp_fDWI_2.nii.gz'}
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_fDWI_2.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_T2Flair.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_fDWI_2.nii.gz
命令执行完毕,耗时4.00秒
命令执行完毕,耗时2.59秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_ADC.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_ADC.nii.gz
命令执行完毕,耗时4.50秒
命令执行完毕,耗时1.60秒
Rigid: G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_ADC_manual.nii.gz→G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_fDWI_2.nii.gz
→G:\Aim_new\JSPH_adc2fdwi\X0876251\20160705-Pre\__rsmp_ADC_manual.nii.gz
命令执行完毕,耗时3.84秒


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:59<00:00, 17.93s/it]

命令执行完毕,耗时1.51秒
